## 수정사항
#### 2021.06.04
* Gripper 자유도 45도로 제한됨에 따라 파지점 방향 수정, 바닥 파지면은 삭제

## set running directory

In [1]:
from __future__ import print_function
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

In [2]:
from pkg.controller.combined_robot import *
from pkg.utils.utils import get_now, try_mkdir


DATA_PATH = os.path.join(os.environ['RNB_PLANNING_DIR'], "data")
try_mkdir(DATA_PATH)

TEST_DATA_PATH = os.path.join(DATA_PATH, "test-pairwise")
try_mkdir(TEST_DATA_PATH)

In [3]:
VISUALIZE = False
TIMEOUT_REACH = 30
TIMEOUT_RETRIEVE = 30
TIMEOUT_SELF = 3
TOOL_RPY = (-np.pi/2,0,0)
SHOW_PERIOD = 0.01

ROBOT_TYPE = RobotType.panda

if ROBOT_TYPE == RobotType.indy7:
    ROBOT_NAME = "indy0"
    TOOL_LINK = "indy0_tcp"
    TOOL_XYZ = (0,0,0.14)
    GRIP_DEPTH = 0.05
    HOME_POSE = (0,0,0,0,0,0)
elif ROBOT_TYPE == RobotType.panda:
    ROBOT_NAME = "panda0"
    TOOL_LINK = "panda0_hand"
    TOOL_XYZ = (0,0,0.112)
    GRIP_DEPTH = 0.03
    HOME_POSE = (0,-0.3,0,-0.5,0,2.5,0)
else:
    raise(NotImplementedError("Unknown robot type"))
    
    
ROBOT_DATA_ROOT = os.path.join(TEST_DATA_PATH, ROBOT_TYPE.name)
try_mkdir(ROBOT_DATA_ROOT)

## init combined robot config

In [4]:

from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, ROBOT_TYPE, None,
                INDY_IP)]
              , connection_list=[False])

from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# # deprecated: s_builder.reset_reference_coord(ref_name="floor")

connection command:
panda0: False


## get ghnd with detected robot config

In [5]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {ROBOT_NAME: ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob, start_rviz=VISUALIZE)
HOME_DICT = list2dict(HOME_POSE, gscene.joint_names)


In [6]:
from pkg.utils.joint_utils import get_tf
shoulder_link = gscene.urdf_content.joint_map[gscene.joint_names[1]].child
shoulder_height = get_tf(shoulder_link, HOME_DICT, gscene.urdf_content)[2,3]

## add environment

In [7]:
from pkg.geometry.geometry import *
gtems_robot = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

## init planning scene

In [8]:
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

## Register binders

In [9]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane, SweepTool

In [10]:
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name=TOOL_LINK, 
                 dims=(0.01,)*3, center=TOOL_XYZ, rpy=TOOL_RPY, color=(1,0,0,1), display=True, collision=False, fixed=True)
gripper = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0))

## planner

In [11]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
from pkg.planning.filtering.grasp_filter import GraspChecker
mplan = MoveitPlanner(pscene)
checker = MoveitPlanner(pscene, motion_filters = [GraspChecker(pscene)])

## ui

In [12]:
from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Object Classes

In [13]:
from pkg.planning.filtering.lattice_model import *
from pkg.planning.filtering.lattice_model.scene_building import *
        
PLANE_THICKNESS = 0.1
CLEARANCE = 1e-3

##
# @class WorkPlane
# @brief working plane. target and obstacle objects are generated on this plane
class WorkPlane(ObstacleBase):
    RTH_MIN = (0.3, -np.pi/2, -0.2)
    RTH_MAX = (0.3, +np.pi/2, 0.5)
    RPY_MIN = (0, 0, -np.pi/6)
    RPY_MAX = (0, 0, +np.pi/6)
    DIM_MIN = (0.6, 0.6, PLANE_THICKNESS)
    DIM_MAX = (0.6, 0.6, PLANE_THICKNESS)
    GTYPE = GEOTYPE.BOX
    COLOR=  (0.8,0.8,0.2,0.5)
    
    def __init__(self, gscene, name, *args, **kwargs):
        ObstacleBase.__init__(self, gscene, name, *args, **kwargs)
        
    def is_overlapped_with(self, gtem):
        verts, radii = gtem.get_vertice_radius()
        verts_global = np.add(np.matmul(verts, gtem.orientation_mat.transpose()), gtem.center)
        verts_wp = np.multiply(DEFAULT_VERT_DICT[self.GTYPE], tuple(self.DIM[:2])+(self.H,))
        verts_wp_global = np.add(np.matmul(verts_wp, self.geometry.orientation_mat.transpose()), 
                                 np.add(self.geometry.center, (0,0,self.H/2)))
        return get_gjk_distance(get_point_list(verts_global), get_point_list(verts_wp_global))-radii < 1e-4
    
##
# @class PlaneObstacle
# @brief Obstacles on the workplane
class PlaneObject(ObstacleBase):
    RTH_MIN = (0.2, -np.pi/2, -0.2)
    RTH_MAX = (0.8, +np.pi/2, +0.5)
    RPY_MIN = (0, 0, 0)
    RPY_MAX = (0, 0, 0)
    DIM_MIN = (0.02, GRIP_DEPTH, GRIP_DEPTH)
    DIM_MAX = (0.055, 0.3, 0.3)
    GTYPE = GEOTYPE.BOX
    COLOR =  (0.2,0.2,0.8,0.5)
    def __init__(self, gscene, name, workplane, GRIP_DEPTH, CLEARANCE, XYZ_LOC=None, **kwargs):
        self.GRIP_DEPTH = GRIP_DEPTH
        self.DIM_MIN = (0.02, GRIP_DEPTH, GRIP_DEPTH)
        ObstacleBase.__init__(self, gscene=gscene, name=name, **kwargs)
        verts, radii = self.geometry.get_vertice_radius()
        verts_rot = np.matmul(self.geometry.orientation_mat, verts.transpose()) ## verices with global orientaion
        verts_rot_loc = np.matmul(workplane.geometry.Toff[:3,:3].transpose(), verts_rot) ## verices with local orientaion
        max_verts = np.max(verts_rot_loc, axis=-1)
        min_verts = np.min(verts_rot_loc, axis=-1)
        if XYZ_LOC is None:
            self.XYZ_LOC = np.random.uniform(np.negative(workplane.DIM)/2-min_verts+radii,np.array(workplane.DIM)/2-max_verts-radii)
            self.XYZ_LOC[2] = workplane.DIM[2]/2 + self.DIM[2]/2 + CLEARANCE
        else:
            self.XYZ_LOC = self.XYZ_LOC
        self.XYZ = np.matmul(workplane.geometry.Toff[:3,:3], self.XYZ_LOC) + workplane.geometry.Toff[:3,3]
        self.RPY = (0,0,0)
        self.geometry.set_offset_tf(center = self.XYZ, orientation_mat=Rot_rpy(self.RPY))
        self.RTH = cart2cyl(*self.XYZ)
        gscene.update_marker(self.geometry)
        
        


            
def disperse_objects(gscene, object_class, key, Nobj, workplane_on, GRIP_DEPTH, CLEARANCE=1e-3):
    clear_class(gscene, key, Nobj)
        
    obs_list = []
    for iw in range(Nobj):
        while len(obs_list) != iw+1:
            obs = object_class(gscene, "{}_{}".format(key, iw), workplane_on, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)
            remove_this = False
            for obs_pre in obs_list:
                if obs_pre.is_overlapped_with(obs.geometry):
                    remove_this = True
                    break
            if remove_this:
                gscene.remove(obs.geometry)
            else:
                obs_list.append(obs)
    return obs_list


def add_object(pscene, obj, HANDLE_THICKNESS=1e-6, HANDLE_COLOR = (1,0,0,0.3)):
    gscene = pscene.gscene
    handles = []
    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_tp_a", link_name="base_link", 
                       dims=(obj.DIM[1], GRIP_DEPTH, HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(np.pi/2,0,np.pi/2), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_tp_b", link_name="base_link", 
                       dims=(obj.DIM[1], GRIP_DEPTH, HANDLE_THICKNESS), center=(0,0,obj.DIM[2]/2-GRIP_DEPTH/2), rpy=(np.pi/2,0,-np.pi/2), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_ft_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_ft_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,obj.DIM[1]/2-GRIP_DEPTH/2,0), rpy=(0,-np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_bk_a", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(-np.pi,-np.pi/2,0), 
                       color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                   parent=obj.name)
    )

    handles.append(
        gscene.create_safe(gtype=GEOTYPE.BOX, name="hdl_bk_b", link_name="base_link", 
                       dims=(obj.DIM[2], GRIP_DEPTH,HANDLE_THICKNESS), center=(0,-obj.DIM[1]/2+GRIP_DEPTH/2,0), rpy=(-np.pi,+np.pi/2,0), 
                           color=HANDLE_COLOR, display=True, collision=False, fixed=False,
                       parent=obj.name)
    )

    action_points_dict = {"placement": PlacePoint("placement", obj.geometry, [0,0,-obj.DIM[2]/2], [0,0,0])}
    grasp_points = {handle.name: Grasp2Point(handle.name, handle, None, (0,0,0)) for handle in handles}
    for k, ap in grasp_points.items():
        ap.get_redundancy = lambda : {'w': (0, 0), 'x': (0, 0), 'y': (0, 0), 'z': (0, 0)}
    action_points_dict.update(grasp_points)
    obj_pscene = pscene.create_subject(oname=obj.name, gname=obj.name, _type=CustomObject, 
                                 action_points_dict=action_points_dict)
    return obj_pscene, handles

In [14]:
N_max_sample = 100
N_print = 5
N_sample_single_obj = 1

In [15]:
from pkg.planning.filtering.pair_svm import PairSVM
pchecker = MoveitPlanner(pscene)
pchecker.motion_filters = [PairSVM(pscene)]

### sampling

In [16]:
gtimer = GlobalTimer.instance()
gtimer.reset()

i_print = 0

for _ in range(10):

    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, get_now())
    try_mkdir(DATASET_PATH)
    print("")
    print("DATASET_PATH: {}".format(DATASET_PATH))
    print("")

    gtimer.tic("full_loop")

    i_print = 0
    i_s = 0
    i_file = 0
    reach_list = []
    retrieve_list = []
    pres_list = []
    while i_s < N_max_sample:
        ## add floor, ceiling
        floor = Floor(gscene, "floor")
        # ceiling = Ceiling(gscene, "ceiling")

        ## set workplane
        wp = WorkPlane(gscene, "wp")
        pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)

        ## add object
        obj_list = disperse_objects(gscene, PlaneObject, "obj", 2, workplane_on=wp, GRIP_DEPTH=GRIP_DEPTH, CLEARANCE=CLEARANCE)

        obj = obj_list[0]

        
        gtem_args = []
        samples = []
        for gtem in gscene:
            if gtem.link_name == "base_link":
                gtem_args.append(deepcopy(gtem.get_args()))
        obj_args = {obj.name: {"GRIP_DEPTH": obj.GRIP_DEPTH, "DIM": obj.DIM, "gname": obj.geometry.name} 
                    for obj in obj_list}

        for obj in obj_list:
            obj_pscene, handles = add_object(pscene, obj)
            checker.update_gscene(only_self_collision=True)
            mplan.update_gscene()
            initial_state = pscene.initialize_state(HOME_POSE)
            pscene.set_object_state(initial_state)
            from_state = initial_state.copy(pscene)
            to_node = ("grip0",)
            available_binding_dict = pscene.get_available_binding_dict(from_state, to_node, HOME_DICT)
            for _ in range(N_sample_single_obj):
                to_state, redundancy_dict = pscene.sample_leaf_state(from_state, available_binding_dict, to_node)
                
                pscene.set_object_state(from_state)
                
                pres = pchecker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                                test_filters_only=True)
                pres_list.append(pres)
                
                gtimer.tic("kin")
                Traj_kin, LastQ, error, success_kin, binding_list = \
                            checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                                    only_self_collision=True, timeout=TIMEOUT_SELF)
                time_kin = gtimer.toc("kin")

                success_reach, success_retrieve = False, False
                time_reach, time_retrieve = TIMEOUT_REACH, TIMEOUT_RETRIEVE
                Traj_reach, Traj_retrieve = [], []
                if success_kin:
                    gtimer.tic("reach")
                    Traj_reach, LastQ, error, success_reach, binding_list = mplan.plan_transition(
                        from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, timeout=TIMEOUT_REACH)
                    time_reach = gtimer.toc("reach")
        #             print("reach: {}".format(success_reach))
                    if success_reach:
                        if VISUALIZE:
                            gscene.show_motion(Traj_reach, period=SHOW_PERIOD)
                        for bd in binding_list:
                            pscene.rebind(bd, list2dict(LastQ, pscene.gscene.joint_names))
                        binding_state, state_param = pscene.get_object_state()
                        new_state = State(binding_state, state_param, list(LastQ), pscene)
                        end_state = new_state.copy(pscene)
                        end_state.Q = np.array(HOME_POSE)
                        gtimer.tic("retrieve")
                        Traj_retrieve, LastQ, error, success_retrieve, binding_list = mplan.plan_transition(
                            from_state=new_state, to_state=end_state, timeout=TIMEOUT_RETRIEVE)
                        time_retrieve = gtimer.toc("retrieve")
        #                 print("retrieve: {}".format(success_retrieve))
                        if success_retrieve and VISUALIZE:
                            gscene.show_motion(Traj_retrieve, period=SHOW_PERIOD)
                else:
                    pass # doen't save pre-filtered data
                
                reach_list.append(success_reach)
                retrieve_list.append(success_retrieve)
                samples.append((obj.name, from_state, to_state, redundancy_dict, 
                                success_kin, success_reach, success_retrieve, 
                                Traj_kin, Traj_reach, Traj_retrieve, 
                                time_kin, time_reach, time_retrieve))
                i_s += 1 
            pscene.set_object_state(initial_state)

            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)    

            gscene.update_markers_all()
            time_elapsed = gtimer.toc("full_loop")/1000
            i_print += 1
            if i_print%N_print == 0:
                print("{} / {} in {} / {} s -- reach,retrieve, pacc = ({} %, {} %, {} %)                     ".format(
                    i_s, N_max_sample, round(time_elapsed, 2), round(time_elapsed/i_s*N_max_sample, 2), 
                    round(np.mean(reach_list)*100, 1), round(np.mean(retrieve_list)*100, 1),
                    round(np.mean(np.equal(retrieve_list, pres_list))*100, 1)
                ), end="\r")
                i_print = 0
                
        i_file += 1
        save_pickle(os.path.join(DATASET_PATH, "%05d.pkl"%i_file), 
                    {"gtem_args":gtem_args, "obj_args": obj_args, "samples": samples})


    print("")
    print("")
    print("============= Finished {} in {} s -- reach,retrieve = ({} %, {} %) =================".format(
        i_s, round(time_elapsed, 2), round(np.mean(reach_list)*100, 1), round(np.mean(retrieve_list)*100, 1)))


DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102211

100 / 100 in 261.6 / 261.6 s -- reach,retrieve, pacc = (36.0 %, 34.0 %, 83.0 %)                      

============= Finished 100 in 261.6 s -- reach,retrieve = (36.0 %, 34.0 %) =================

DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633

100 / 100 in 214.0 / 214.0 s -- reach,retrieve, pacc = (33.0 %, 32.0 %, 88.0 %)                      

============= Finished 100 in 214.0 s -- reach,retrieve = (33.0 %, 32.0 %) =================

DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007

100 / 100 in 172.96 / 172.96 s -- reach,retrieve, pacc = (37.0 %, 37.0 %, 81.0 %)                     

============= Finished 100 in 172.96 s -- reach,retrieve = (37.0 %, 37.0 %) =================

DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103300

100 / 100 in 219.99 / 219.99 s -- reach,retrieve, pacc

# Load and verify

In [17]:
gtimer = GlobalTimer.instance()
gtimer.reset()
DATASET_LIST = sorted(os.listdir(ROBOT_DATA_ROOT))
ERROR_LIST = []
N_tot = 0
N_suc = 0
print("DATASET_LIST: {}".format(DATASET_LIST))
for DATASET_DIR in DATASET_LIST:
    DATASET_PATH = os.path.join(ROBOT_DATA_ROOT, DATASET_DIR)
    print("DATASET_PATH: {}".format(DATASET_PATH))
    DATA_LIST = sorted(os.listdir(DATASET_PATH))
    print("DATA_LIST: {}".format(DATA_LIST))
    for DATA_FILE in DATA_LIST:
        DATA_PATH = os.path.join(DATASET_PATH, DATA_FILE)
        print("DATA_PATH: {}".format(DATA_PATH))
        data = load_pickle(DATA_PATH)
        gtem_args = data["gtem_args"]
        obj_args = data["obj_args"]
        samples = data["samples"]
        gtem_remove = []
        for gtem in gscene:
            if gtem.link_name == "base_link" and gtem.parent is None:
                gtem_remove.append(gtem)
        for gtem in gtem_remove:
            gscene.remove(gtem)
                
        gid_list = np.arange(len(gtem_args)).tolist()
        for gidx in gid_list:
            args = gtem_args[gidx]
            if args['parent'] is not None:
                if args['parent'] not in gscene.NAME_DICT:
                    gid_list.append(gidx)
                    continue
            gscene.create_safe(**args)
            
        pscene.create_binder(bname="wp", gname="wp", _type=PlacePlane, point=None)
            
        for i_s, sample in enumerate(samples):
            obj_name, from_state, to_state, redundancy_dict, \
                success_kin_, success_reach_, success_retrieve_, \
                Traj_kin_, Traj_reach_, Traj_retrieve_, \
                time_kin_, time_reach_, time_retrieve_ = sample
            obj, obj_arg = DummyObject(), obj_args[obj_name]
            obj.name = obj_name
            obj.GRIP_DEPTH = obj_arg["GRIP_DEPTH"]
            obj.DIM = obj_arg["DIM"]
            obj.geometry = gscene.NAME_DICT[obj_arg["gname"]]
            obj_pscene, handles = add_object(pscene, obj)
            checker.update_gscene(only_self_collision=True)
            mplan.update_gscene()
                            
            pscene.set_object_state(from_state)

            with gtimer.block("kin"):
                Traj_kin, LastQ, error, success_kin, binding_list = \
                    checker.plan_transition(from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, 
                                                only_self_collision=True, timeout=TIMEOUT_SELF)

            success_reach, success_retrieve = False, False
            time_reach, time_retrieve = TIMEOUT_REACH, TIMEOUT_RETRIEVE
            Traj_reach, Traj_retrieve = [], []
            if success_kin:
                with gtimer.block("reach"):
                    Traj_reach, LastQ, error, success_reach, binding_list = mplan.plan_transition(
                        from_state=from_state, to_state=to_state, redundancy_dict=redundancy_dict, timeout=TIMEOUT_REACH)
                    
                if success_reach:
                    if VISUALIZE:
                        gscene.show_motion(Traj_reach, period=SHOW_PERIOD)
                    for bd in binding_list:
                        pscene.rebind(bd, list2dict(LastQ, pscene.gscene.joint_names))
                    binding_state, state_param = pscene.get_object_state()
                    new_state = State(binding_state, state_param, list(LastQ), pscene)
                    end_state = new_state.copy(pscene)
                    end_state.Q = np.array(HOME_POSE)
                    
                    with gtimer.block("retrieve"):
                        Traj_retrieve, LastQ, error, success_retrieve, binding_list = mplan.plan_transition(
                            from_state=new_state, to_state=end_state, timeout=TIMEOUT_RETRIEVE)
    #                 print("retrieve: {}".format(success_retrieve))
                    if success_retrieve and VISUALIZE:
                        gscene.show_motion(Traj_retrieve, period=SHOW_PERIOD)
            else:
                pass # doen't save pre-filtered data
                
            pscene.set_object_state(from_state)
            pscene.remove_subject(obj_pscene.oname)
            for handle in handles:
                gscene.remove(handle)  
            gscene.update_markers_all()  
            N_tot += 1
            N_suc += success_retrieve_
            
            if success_kin_ != success_kin:
                ERROR_LIST.append((DATA_PATH, i_s, "kinematic result changed {} -> {}".format(success_kin_,success_kin)))
                continue
            if success_reach_ != success_reach:
                ERROR_LIST.append((DATA_PATH, i_s, "reach result changed {} -> {}".format(success_reach_,success_reach)))
                continue
            if success_retrieve_ != success_retrieve:
                ERROR_LIST.append((DATA_PATH, i_s, "retreive result changed {} -> {}".format(success_retrieve_,success_retrieve)))
                continue
        print("ERRORS: {} / SUCCESS: {} / TOTAL: {}".format(len(ERROR_LIST), N_suc, N_tot))

DATASET_LIST: ['20210611-102211', '20210611-102633', '20210611-103007', '20210611-103300', '20210611-103640', '20210611-104240', '20210611-104606', '20210611-104954', '20210611-105222', '20210611-105601']
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102211
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl', '00020.pkl', '00021.pkl', '00022.pkl', '00023.pkl', '00024.pkl', '00025.pkl', '00026.pkl', '00027.pkl', '00028.pkl', '00029.pkl', '00030.pkl', '00031.pkl', '00032.pkl', '00033.pkl', '00034.pkl', '00035.pkl', '00036.pkl', '00037.pkl', '00038.pkl', '00039.pkl', '00040.pkl', '00041.pkl', '00042.pkl', '00043.pkl', '00044.pkl', '00045.pkl', '00046.pkl', '00047.pkl', '00048.pkl', '00049.pkl', '00050.pkl']
DATA_PATH: /home/rnb/Projects/rnb-planning/dat

ERRORS: 1 / SUCCESS: 35 / TOTAL: 102
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633/00002.pkl
ERRORS: 1 / SUCCESS: 37 / TOTAL: 104
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633/00003.pkl
ERRORS: 1 / SUCCESS: 37 / TOTAL: 106
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633/00004.pkl
ERRORS: 1 / SUCCESS: 37 / TOTAL: 108
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633/00005.pkl
ERRORS: 1 / SUCCESS: 37 / TOTAL: 110
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633/00006.pkl
ERRORS: 1 / SUCCESS: 37 / TOTAL: 112
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633/00007.pkl
ERRORS: 1 / SUCCESS: 37 / TOTAL: 114
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633/00008.pkl
ERRORS: 1 / SUCCESS: 38 / TOTAL: 116
DATA_PATH: /home/rnb/Projects/rnb-planning/dat

ERRORS: 2 / SUCCESS: 75 / TOTAL: 216
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00009.pkl
ERRORS: 3 / SUCCESS: 75 / TOTAL: 218
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00010.pkl
ERRORS: 3 / SUCCESS: 76 / TOTAL: 220
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00011.pkl
ERRORS: 3 / SUCCESS: 77 / TOTAL: 222
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00012.pkl
ERRORS: 3 / SUCCESS: 77 / TOTAL: 224
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00013.pkl
ERRORS: 3 / SUCCESS: 78 / TOTAL: 226
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00014.pkl
ERRORS: 3 / SUCCESS: 79 / TOTAL: 228
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00015.pkl
ERRORS: 3 / SUCCESS: 80 / TOTAL: 230
DATA_PATH: /home/rnb/Projects/rnb-planning/dat

ERRORS: 4 / SUCCESS: 112 / TOTAL: 330
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103300/00016.pkl
ERRORS: 4 / SUCCESS: 112 / TOTAL: 332
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103300/00017.pkl
ERRORS: 4 / SUCCESS: 113 / TOTAL: 334
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103300/00018.pkl
ERRORS: 4 / SUCCESS: 114 / TOTAL: 336
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103300/00019.pkl
ERRORS: 4 / SUCCESS: 114 / TOTAL: 338
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103300/00020.pkl
ERRORS: 4 / SUCCESS: 115 / TOTAL: 340
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103300/00021.pkl
ERRORS: 4 / SUCCESS: 115 / TOTAL: 342
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103300/00022.pkl
ERRORS: 4 / SUCCESS: 116 / TOTAL: 344
DATA_PATH: /home/rnb/Projects/rnb-plan

ERRORS: 4 / SUCCESS: 154 / TOTAL: 444
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103640/00023.pkl
ERRORS: 4 / SUCCESS: 155 / TOTAL: 446
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103640/00024.pkl
ERRORS: 4 / SUCCESS: 155 / TOTAL: 448
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103640/00025.pkl
ERRORS: 4 / SUCCESS: 156 / TOTAL: 450
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103640/00026.pkl
ERRORS: 4 / SUCCESS: 156 / TOTAL: 452
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103640/00027.pkl
ERRORS: 4 / SUCCESS: 157 / TOTAL: 454
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103640/00028.pkl
ERRORS: 4 / SUCCESS: 157 / TOTAL: 456
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103640/00029.pkl
ERRORS: 4 / SUCCESS: 158 / TOTAL: 458
DATA_PATH: /home/rnb/Projects/rnb-plan

ERRORS: 4 / SUCCESS: 195 / TOTAL: 558
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00030.pkl
ERRORS: 4 / SUCCESS: 196 / TOTAL: 560
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00031.pkl
ERRORS: 4 / SUCCESS: 196 / TOTAL: 562
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00032.pkl
ERRORS: 4 / SUCCESS: 198 / TOTAL: 564
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00033.pkl
ERRORS: 4 / SUCCESS: 199 / TOTAL: 566
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00034.pkl
ERRORS: 4 / SUCCESS: 199 / TOTAL: 568
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00035.pkl
ERRORS: 4 / SUCCESS: 199 / TOTAL: 570
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00036.pkl
ERRORS: 4 / SUCCESS: 201 / TOTAL: 572
DATA_PATH: /home/rnb/Projects/rnb-plan

ERRORS: 7 / SUCCESS: 232 / TOTAL: 672
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104606/00037.pkl
ERRORS: 7 / SUCCESS: 233 / TOTAL: 674
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104606/00038.pkl
ERRORS: 7 / SUCCESS: 234 / TOTAL: 676
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104606/00039.pkl
ERRORS: 7 / SUCCESS: 235 / TOTAL: 678
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104606/00040.pkl
ERRORS: 7 / SUCCESS: 236 / TOTAL: 680
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104606/00041.pkl
ERRORS: 7 / SUCCESS: 236 / TOTAL: 682
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104606/00042.pkl
ERRORS: 7 / SUCCESS: 236 / TOTAL: 684
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104606/00043.pkl
ERRORS: 7 / SUCCESS: 238 / TOTAL: 686
DATA_PATH: /home/rnb/Projects/rnb-plan

ERRORS: 7 / SUCCESS: 277 / TOTAL: 786
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104954/00044.pkl
ERRORS: 7 / SUCCESS: 277 / TOTAL: 788
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104954/00045.pkl
ERRORS: 7 / SUCCESS: 278 / TOTAL: 790
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104954/00046.pkl
ERRORS: 7 / SUCCESS: 279 / TOTAL: 792
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104954/00047.pkl
ERRORS: 7 / SUCCESS: 279 / TOTAL: 794
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104954/00048.pkl
ERRORS: 7 / SUCCESS: 281 / TOTAL: 796
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104954/00049.pkl
ERRORS: 7 / SUCCESS: 282 / TOTAL: 798
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104954/00050.pkl
ERRORS: 7 / SUCCESS: 282 / TOTAL: 800
DATASET_PATH: /home/rnb/Projects/rnb-p

ERRORS: 8 / SUCCESS: 323 / TOTAL: 900
DATASET_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-105601
DATA_LIST: ['00001.pkl', '00002.pkl', '00003.pkl', '00004.pkl', '00005.pkl', '00006.pkl', '00007.pkl', '00008.pkl', '00009.pkl', '00010.pkl', '00011.pkl', '00012.pkl', '00013.pkl', '00014.pkl', '00015.pkl', '00016.pkl', '00017.pkl', '00018.pkl', '00019.pkl', '00020.pkl', '00021.pkl', '00022.pkl', '00023.pkl', '00024.pkl', '00025.pkl', '00026.pkl', '00027.pkl', '00028.pkl', '00029.pkl', '00030.pkl', '00031.pkl', '00032.pkl', '00033.pkl', '00034.pkl', '00035.pkl', '00036.pkl', '00037.pkl', '00038.pkl', '00039.pkl', '00040.pkl', '00041.pkl', '00042.pkl', '00043.pkl', '00044.pkl', '00045.pkl', '00046.pkl', '00047.pkl', '00048.pkl', '00049.pkl', '00050.pkl']
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-105601/00001.pkl
ERRORS: 8 / SUCCESS: 323 / TOTAL: 902
DATA_PATH: /home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-1056

In [18]:
print(len(ERROR_LIST))
for ERROR in ERROR_LIST:
    print(ERROR)

8
('/home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102211/00011.pkl', 1, 'reach result changed True -> False')
('/home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-102633/00013.pkl', 0, 'reach result changed False -> True')
('/home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00009.pkl', 1, 'reach result changed False -> True')
('/home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-103007/00037.pkl', 0, 'reach result changed True -> False')
('/home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00038.pkl', 0, 'reach result changed False -> True')
('/home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104240/00048.pkl', 0, 'reach result changed True -> False')
('/home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-104606/00003.pkl', 1, 'kinematic result changed False -> True')
('/home/rnb/Projects/rnb-planning/data/test-pairwise/panda/20210611-105222/00007.pkl', 0, 'reach r

* 100 개에 인디는 1분, 판다는 2~3분 수준, 각 1000개는 30분 정도
* 이형 데이터 x 2 : 40 분

['20210611-050055']

## ====================== deprecated ============================

## prepare cells

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
with gtimer.block("prepare_reference_cells_ltc_full"):
    ltc_full.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_effector"):
    ltc_effector.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_arm_05"):
    ltc_arm_05.prepare_reference_cells(gscene)
with gtimer.block("prepare_reference_cells_ltc_arm_10"):
    ltc_arm_10.prepare_reference_cells(gscene)
print(gtimer)

In [ ]:
gscene.show_motion(Traj_reach)

## visualize

In [ ]:
vistem_list = []
for col_idx in grasp_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in grasp_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

In [ ]:
vistem_list = []
for col_idx in arm_10_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in arm_10_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

In [ ]:
vistem_list = []
for col_idx in full_scene_dict['tar']:
    vistem_list.append(gscene.copy_from(ltc_full.cell_refs[col_idx], color=(0,0,1,0.3)))

for col_idx in full_scene_dict['tool']:
    vistem_list.append(gscene.copy_from(ltc_full.cell_refs[col_idx], color=(0,1,0,0.3)))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## grasp conversion

In [ ]:
gtimer.reset()
with gtimer.block("convert_vertices"):
    ltc_effector.convert_vertices(actor_vertinfo_list, HOME_DICT, Tref=Tref)
    ltc_effector.convert_vertices(object_vertinfo_list, HOME_DICT, Tref=Tref)
print(gtimer)

In [ ]:
vistem_list = []
for coll_idxes in ltc_effector.coll_idx_dict.values():
    for col_idx in coll_idxes:
        vistem_list.append(gscene.copy_from(ltc_effector.cell_refs[col_idx]))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## scene conversion

In [ ]:
gtimer.reset()
with gtimer.block("ltc_arm_10_convert"):
    Tref_base = SE3(Tref[:3,:3], (0, 0, shoulder_height))
    ltc_arm_10.convert([gtem for gtem in gscene if gtem.collision and gtem not in gtems_robot], HOME_DICT, Tref=Tref_base)
print(gtimer)

In [ ]:
vistem_list = []
for coll_idxes in ltc_arm_10.coll_idx_dict.values():
    for col_idx in coll_idxes:
        vistem_list.append(gscene.copy_from(ltc_arm_10.cell_refs[col_idx]))

In [ ]:
for vistem in vistem_list:
    if vistem in gscene:
        gscene.remove(vistem)

## remove object

In [ ]:

pscene.remove_object(obj_pscene.oname)
for handle in handles:
    gscene.remove(handle)

In [ ]:
gscene.get_items_on_links(TOOL_LINK_BUNDLE[0]) + gscene.get_items_on_links(TOOL_LINK_BUNDLE[1])

In [ ]:
with gtimer.block("convert"):
    ltc_full.convert([gtem for gtem in gscene if gtem not in gtems_robot], crob.home_dict)

In [ ]:
print(gtimer)

In [ ]:
gscene.show_pose(crob.home_pose)

In [ ]:
# for coll_idxes in latticizer.coll_idx_dict.values():
#     for col_idx in coll_idxes:
#         gscene.copy_from(latticizer.cell_refs[col_idx])

## deprecated python ver

In [ ]:
from pkg.utils.gjk import make_point3, get_point_list, get_point_list_list, get_point_list_list_from_point_list
from pkg.utils.gjk import get_gjk_distance, get_gjk_distance_min, get_gjk_distance_all

In [ ]:
gtimer = GlobalTimer.instance()
gtimer.reset()
with gtimer.block("get_cell_vertices"):
    gscene_ref = GeometryScene(gscene.urdf_content, gscene.urdf_path, gscene.joint_names, gscene.link_names, rviz=False)
    centers = get_centers(Nwdh, L_CELL, OFFSET_ZERO).reshape((-1,3))
    cell_refs = []
    for icell, center in enumerate(centers):
        cell_refs.append(gscene_ref.create_safe(GEOTYPE.BOX, str(icell), "base_link", dims=(L_CELL,)*3, 
                                                center=center, rpy=(0,0,0), color=(1,1,1,0.2), 
                                                display=True, collision=False, fixed=True))
    cell_vertices = get_cell_vertices(centers, L_CELL)
    cell_vertices_gjk = get_point_list_list(cell_vertices)
    center_vertices_gjk = get_point_list_list(np.expand_dims(centers, axis=-2))

In [ ]:
print(gtimer)

In [ ]:
gtimer.reset()
sqrt3 = np.sqrt(3)
LCmax = L_CELL*sqrt3/2
Qdict = crob.home_dict
with gtimer.block("full"):
    for gtem in gscene:
#         if gtem in gtems_robot or not gtem.collision:
#             continue
        with gtimer.block("centers"):
            Tgtem = SE3_inv(gtem.get_tf(Qdict))
            centers_loc = np.abs(np.matmul(centers, Tgtem[:3,:3].transpose())+Tgtem[:3,3])
        if gtem.gtype == GEOTYPE.BOX and np.sum(np.abs(gtem.rpy))<1e-5:
            with gtimer.block("right box"):
                dist_list = np.max(centers_loc - np.divide(gtem.dims,2) - L_CELL, axis=-1)
                cell_idx_occupy = np.where(dist_list < 1e-3)[0]
                continue
        with gtimer.block("calc_center_dist"):
            dist_list = np.max(centers_loc - np.divide(gtem.dims,2) - LCmax, axis=-1)
            idx_candi = np.where(dist_list<0)[0]
        with gtimer.block("calc_points"):
            gtem_verts, gtem_radius = gtem.get_vertice_radius_from(Qdict)
        with gtimer.block("get_point_gjk"):
            gtem_verts_gjk = get_point_list(gtem_verts)
        with gtimer.block("get_distance_gjk"):
            cell_idx_occupy = []
            for idx in idx_candi:
                cell_candi = cell_vertices_gjk[idx]
                center_v_candi = center_vertices_gjk[idx]
                if ((get_gjk_distance(cell_candi, gtem_verts_gjk) - gtem_radius < 1e-4)):
                    cell_idx_occupy.append(idx)
#                 if ((get_gjk_distance(gtem_verts_gjk, center_v_candi) - gtem_radius - L_CELL/2 < 1e-4) or 
#                     (get_gjk_distance(cell_candi, gtem_verts_gjk) - gtem_radius < 1e-4)):
#                     cell_idx_occupy.append(idx)
print(gtimer)